In [1]:
import pandas as pd
import torch
import re

In [2]:
def filter_arabic_only(text):
    if not isinstance(text, str):
        return ""
    return re.sub(r'[^\u0600-\u06FF\s0-9]', '', text)

In [3]:
datasetsCount = 1
combinedText = []
datasetsFeatures = ["text","story"]
for i in range(datasetsCount):
    df = pd.read_csv(f"../Data/RAW_dataset/ArabicLLM{i + 1}")
    df = df.dropna()
    df = df.drop_duplicates()
    df[datasetsFeatures[i]] = df[datasetsFeatures[i]].astype(str)
    df[datasetsFeatures[i]] = df[datasetsFeatures[i]].apply(filter_arabic_only)
    df[datasetsFeatures[i]] = df[datasetsFeatures[i]]
    
   
    for j in df[datasetsFeatures[i]]:
        combinedText.append(j)

<class 'str'>


In [13]:
combinedText = combinedText[:100000]

In [14]:
len(combinedText)

100000

In [20]:


def clean_arabic_for_tiny_model(text):
    if not text or not isinstance(text, str):
        return ""
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'ـ', '', text)
    text = re.sub(r'[\u064B-\u065F]', '', text)
    text = re.sub(r'[أإآ]', 'ا', text)
    numbersMapping = str.maketrans('٠١٢٣٤٥٦٧٨٩', '0123456789')
    text = text.translate(numbersMapping)
    text = text.replace(',', '،').replace('?', '؟')
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def is_valid_line(text, min_words=20):
    if len(text.split()) < min_words:
        return False
    return True

cleanedText = []
for i in combinedText:
    if is_valid_line(i):
        text = clean_arabic_for_tiny_model(i) + " <|endoftext|> "
        cleanedText.append(text)
        

combinedText = cleanedText

In [23]:
count = 0
for i in combinedText:
    if len(i.split()) <= 20:
        count = count + 1
print(count)

142


In [37]:
combinedText[5]

'اخواني الاعضاء والزوار الشبكة في خدمة الجميع اذا عندك اعلان ارسل بيانات الاعلان والصور ورقم الجوال وسوف نقوم بوضع الاعلان باسمك في القسم المخصص له وهذا رقم الواتساب 054458211 التسجيل البحث مشاركات اليوم اجعل كافة الاقسام مقروءة اهلا وسهلا بك الى شبكة العود الاول اهلا وسهلا بك زائرنا الكريم اذا كانت هذه زيارتك الاولى للمنتدى فيرجى التكرم بزيارة صفحة التعليم ات بالضغط هنا كما يشرفنا ان تقوم بالتسجيل بالضغط هنا اذا رغبت بالمشاركة في المنتدى اما اذا رغبت بقراءة المواضيع والاطلاع فتفضل بزيارة القسم الذي ترغب ادناه 2020 64 الكات ب مشاركات 0 المشاهدات 1 الوقت 08 23 التاريخ 10 16 2021 2020 3 18 الكات ب مشاركات 0 المشاهدات 1 الوقت 08 19 التاريخ 10 16 2021 7 2 0 الكات ب مشاركات 0 المشاهدات 1 الوقت 08 16 التاريخ 10 16 2021 5 0 الكات ب مشاركات 0 المشاهدات 1 الوقت 08 12 التاريخ 10 16 2021 5 0 2020 الكات ب مشاركات 0 المشاهدات 1 الوقت 08 08 التاريخ 10 16 2021 2020 8 الكات ب مشاركات 0 المشاهدات 1 الوقت 08 05 التاريخ 10 16 2021 2020 8 2016 الكات ب مشاركات 0 المشاهدات 1 الوقت 08 02 التاريخ 10 16 2021 3

In [24]:
def countOfWords(lisOFWords:list) -> int:
    counter = 0
    for i in lisOFWords:
        text = i.split()
        counter = len(text) + counter
    return counter

In [25]:
countOfWords(cleanedText)

68777574

In [9]:
combinedText[0]

'الخيار الثنائي 2017 تداول الخيارات الثنائية تكتسب شعبية بسرعة منذ ادخالها في عام وسيساعدك ايجاد خدمة الاشارات الجيدة لتداول الخيارات الثنائية في ضمان نجاحك كمضارب ثنائية خيارات استراتيجية سهلة انت هنا الرئيسية 0 0 0 وشهد الرائدة تجار الخيارات الثنائية تغطي الخيارات الثنائية الفوركس لا يزال متسقا في الاسهم قيمة حاسبة معلومات فروكسروس على الذهب يامانا يتم تحديثها بشكل مستمر خلال استراتيجية التداول والمواد فبركسبروس في فكسو كيفية البدء في تقديم خيارات ثنائية ناجحة واختيار الاستراتيجية الصحيحة وسيط والادوات تثبيت وتنشيط من قبل المستخدم كيفية الدخول الى 10٪ من التجار الذين يعملون باستمرار زائد لم يكن جعل الخيارات الثنائية المال من اجل البدء في تداول تحتاج الى انشاء حساب تداول اولا يرجى اتباع الخطوات التالية انقر على زر رصيد الحساب التي تظهر على اسفل الجانب الايسر انقر على وديعة حدد طريقة الايداع التي تفضلها كيف ستظهر المسحوبات في كشف حساب 2945 11861 11923 10506 1849 10292 6701 10995 6569 11769 بدات براس مال 200 ريال وزادت المبيعات الى 400 ريال بعد 200 40 00 استعراض المستغل الروبوت الرئيسي 

In [ ]:
from transformers import AutoTokenizer


tokenizerGPT = AutoTokenizer.from_pretrained("gpt2")

customTokenizer = tokenizerGPT.train_new_from_iterator(combinedText, vocab_size=16000,new_special_tokens=["<QUESTION>","<ANSWER>","<|endoftext|>","[PAD]"],
                                                       initial_alphabet=[])

customTokenizer.add_special_tokens({'pad_token': '[PAD]'})

customTokenizer.save_pretrained("./TOKENIZER")

('./TOKENIZER\\tokenizer_config.json',
 './TOKENIZER\\special_tokens_map.json',
 './TOKENIZER\\vocab.json',
 './TOKENIZER\\merges.txt',
 './TOKENIZER\\added_tokens.json',
 './TOKENIZER\\tokenizer.json')

In [41]:
import shutil
import os

save_path = "./TOKENIZER/"

if os.path.exists(save_path):
    shutil.rmtree(save_path)
    print("Deleted")

os.makedirs(save_path, exist_ok=True)
customTokenizer.save_pretrained(save_path)
print("Saved")

Deleted
Saved


In [42]:
retrivedTokenizer = AutoTokenizer.from_pretrained("./TOKENIZER/")

In [43]:
ids = customTokenizer.encode("بوضع".strip())
decoded = customTokenizer.decode(ids)
print(ids)
print("Decoded Text:", decoded)


[4373, 865]
Decoded Text: بوضع


In [40]:
print("Decoded Text:", customTokenizer.decode(4373))

Decoded Text: بو


In [44]:

full_tokens = retrivedTokenizer(combinedText, truncation=False, padding=False)["input_ids"]


flattened_ids = [token for sequence in full_tokens for token in sequence]
block_size = 512
total_len = (len(flattened_ids) // block_size) * block_size 
chunks = [flattened_ids[i : i + block_size] for i in range(0, total_len, block_size)]

IDs = torch.tensor(chunks)
masks = torch.ones_like(IDs)

Token indices sequence length is longer than the specified maximum sequence length for this model (1282 > 1024). Running this sequence through the model will result in indexing errors


In [15]:
# IDs = outputTokenizer["input_ids"]

# masks = outputTokenizer["attention_mask"]

In [45]:
torch.save(IDs,"../Data/preproccessedData.pt")
torch.save(masks,"../Data/preproccessedDataMASKS.pt")

tokenCOUNT = len(masks) * 512
tokenCOUNT

97325056